In [ ]:
import numpy as np
import csv
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import math
import seaborn as sns

from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, ReLU
from keras.layers import AveragePooling2D
from keras import regularizers
from tensorflow.keras.utils import to_categorical
from keras.callbacks import LearningRateScheduler
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.callbacks import LambdaCallback

data_aug = np.load('/content/drive/My Drive/dataset/data_aug.npy', allow_pickle=False, fix_imports=False)
data_label_aug = np.load('/content/drive/My Drive/dataset/data_label_aug.npy', allow_pickle=False, fix_imports=False)

data_test = np.load('/content/drive/My Drive/dataset/data_test.npy', allow_pickle=False, fix_imports=False)
data_test_label = np.load('/content/drive/My Drive/dataset/data_test_label.npy', allow_pickle=False, fix_imports=False)


train_data = data_aug #[0:88920]
train_labels = data_label_aug #[0:88920]

test_data = data_test
test_labels = data_test_label

LABELS = 'beach bus cafe/restaurant car city_center forest_path grocery_store home library metro_station office park residential_area train tram'.split() #Метки для распознавания
NUM_CLASSES = len(LABELS)

# векторизируем метки
train_labels = to_categorical(train_labels, num_classes = NUM_CLASSES)
test_labels = to_categorical(test_labels, num_classes = NUM_CLASSES)

print('train_data.shape', train_data.shape)
print('test_data.shape', test_data.shape)
print()

print('train_labels.shape', train_labels.shape)
print('test_labels.shape', test_labels.shape)
print('----------------------')
print()



In [ ]:
def deep_cnn(features_shape, num_classes): #Создаём модель сети
    numF = 32

    x = Input(name='inputs', shape=features_shape, dtype='float32')
    o = x
    o = BatchNormalization(name='block1_norm')(o)
    
    o = Conv2D(numF, (3, 3), padding='same', activation='relu', name='block1_conv', kernel_regularizer = regularizers.l2(0.00025), input_shape=features_shape)(o)
    o = BatchNormalization(name='block2_norm')(o)
    o = Conv2D(numF, (3, 3), padding='same', activation='relu', name='block2_conv', kernel_regularizer = regularizers.l2(0.00025))(o)
    o = BatchNormalization(name='block3_norm')(o)
    
    o = MaxPooling2D((3, 3), strides=(2,2), padding='same', name='block1_pool')(o)
    
    o = Conv2D(2*numF, (3, 3), padding='same', activation='relu',  name='block3_conv', kernel_regularizer = regularizers.l2(0.00025))(o)
    o = BatchNormalization(name='block4_norm')(o)
    o = Conv2D(2*numF, (3, 3), padding='same', activation='relu',  name='block4_conv', kernel_regularizer = regularizers.l2(0.00025))(o)
    o = BatchNormalization(name='block5_norm')(o)
    
    o = MaxPooling2D((3, 3), strides=(2,2), padding='same', name='block2_pool')(o)
    
    o = Conv2D(4*numF, (3, 3), padding='same', activation='relu',  name='block5_conv', kernel_regularizer = regularizers.l2(0.00025))(o)
    o = BatchNormalization(name='block6_norm')(o)
    o = Conv2D(4*numF, (3, 3), padding='same', activation='relu',  name='block6_conv', kernel_regularizer = regularizers.l2(0.00025))(o)
    o = BatchNormalization(name='block7_norm')(o)
    
    o = MaxPooling2D((3, 3), strides=(2,2), padding='same', name='block3_pool')(o)
    
    o = Conv2D(8*numF, (3, 3), padding='same', activation='relu',  name='block7_conv', kernel_regularizer = regularizers.l2(0.00025))(o)
    o = BatchNormalization(name='block8_norm')(o)
    o = Conv2D(8*numF, (3, 3), padding='same', activation='relu',  name='block8_conv', kernel_regularizer = regularizers.l2(0.00025))(o)
    o = BatchNormalization(name='block9_norm')(o)
    
    o = AveragePooling2D(pool_size =(16,6))(o)
    o = Dropout(0.5)(o)
    o = Flatten()(o)
    o = Dense(15, activation='softmax', name='pred', kernel_regularizer = regularizers.l2(0.005))(o)

    Model(inputs=x, outputs=o).summary()
    
    return Model(inputs=x, outputs=o)


INPUT_SHAPE = (128,42,2) #Размер входной матрицы. Зависит от параметров спектрограммы
BATCH = 128 #Размер пачки
EPOCHS = 12 #Кол-во эпох
LABELS = 'beach bus cafe/restaurant car city_center forest_path grocery_store home library metro_station office park residential_area train tram'.split() #Метки для распознавания
NUM_CLASSES = len(LABELS)


print('train_data.shape', train_data.shape)
print('test_data.shape', test_data.shape)
print()

print('train_labels.shape', train_labels.shape)
print('test_labels.shape', test_labels.shape)
print('----------------------')
print()
 


sgd = SGD(lr=0.03, momentum=0.8, decay = 0.0, nesterov = False)

model = deep_cnn(INPUT_SHAPE, NUM_CLASSES) #Загружаем созданную модель
model.compile(optimizer=sgd, 
              loss='categorical_crossentropy', 
              metrics=['acc']) #Выбор алгоритма оптимизации и функции ошибки

def scheduler(epoch):
    if epoch < 2:
        return 0.03
    elif epoch >= 2 and epoch < 4:
        return 0.03*0.2
    elif epoch >= 4 and epoch < 6:
        return 0.03*0.2*0.2
    elif epoch >= 6 and epoch < 8:
        return 0.03*0.2*0.2
    elif epoch >= 8 and epoch < 15:
        return 0.03*0.2*0.2*0.2

from tensorflow import keras


my_callbacks = [LearningRateScheduler(scheduler),
                EarlyStopping(monitor='val_acc', patience=4)] #Рання остановка  mode='max', restore_best_weights=True

test_images = test_data

history = model.fit(train_data,
                    train_labels,
                    epochs = 15,
                    batch_size = BATCH,
                    shuffle = True,
                    verbose = 0,
                    callbacks = my_callbacks,
                    validation_data = (test_data, test_labels))

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='Потери на этапе обучения')
plt.plot(epochs, val_loss, 'b', label='Потери на этапе проверки')
plt.title('Потери обучения и проверки')
plt.xlabel('Эпоха')
plt.ylabel('Потери')
plt.legend()

plt.show()

history_dict = history.history

plt.clf()
acc = history.history['acc']
val_acc = history.history['val_acc']

plt.plot(epochs, history.history['acc'], 'bo', label = 'Точность на этапе обучени' )
plt.plot(epochs, history.history['val_acc'], 'b', label = 'Точность на этапе проверки')
plt.title('Точность обучающих и проверочных данных')
plt.xlabel('Эпоха')
plt.ylabel('Точность')
plt.legend()

plt.show()

print()
for i in range (0,len(loss)):
    print('loss')
    print(i+1, ' эпоха: ', loss[i], sep='')
print()

for i in range (0,len(loss)):
    print('val_loss')
    print(i+1, ' эпоха: ', val_loss[i], sep='')
print()

for i in range (0,len(acc)):
    print('acc')
    print(i+1, ' эпоха: ', acc[i], sep='')
print()

for i in range (0,len(acc)):
    print('val_acc')
    print(i+1, ' эпоха: ', val_acc[i], sep='')



test_images = test_data
y_true = data_test_label
classes = LABELS
y_pred = model.predict(test_images)
y_pred = np.argmax(y_pred, axis=1)
con_mat = tf.math.confusion_matrix(labels=y_true, predictions=y_pred).numpy()

con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)
 
con_mat_df = pd.DataFrame(con_mat_norm,
                     index = classes, 
                     columns = classes)

plt.clf()
figure = plt.figure(figsize=(8, 8))
sns.heatmap(con_mat_df, annot=True, cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('Правильные метки')
plt.xlabel('Проверочные метки')
plt.show()